# 1. 로그인

In [1]:
import time
import pandas as pd
from pykiwoom.kiwoom import *
import warnings
warnings.filterwarnings('ignore')

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [2]:
# login - information
account_num = kiwoom.GetLoginInfo('ACCOUNT_CNT') # number of account
accounts = kiwoom.GetLoginInfo('ACCNO') # list of account
user_id = kiwoom.GetLoginInfo('USER_ID') # user id
user_name = kiwoom.GetLoginInfo('USER_NAME') # user name
print('number of account : {}'.format(account_num))

number of account : 1


In [3]:
state = kiwoom.GetConnectState()
if state == 0:
    print("미연결")
elif state == 1:
    print("연결완료")

연결완료


# 2. 변수 설정

In [4]:
from datetime import date,datetime
today = date.today()
today = today.isoformat()
today = today.replace('-','')
yesterday = int(today)-1
yesterday = str(yesterday)
print(yesterday, type(yesterday))
set_d = yesterday

20220707 <class 'str'>


In [5]:
#SectorList : {sector_name: stockcode_list}
sector = {'Food':['097950', '271560', '000080','004370','005300'],
          'Clothing' : ['383220','093050','020000','105630','001070'],
          'Chemical' : ['051910','096770','010950','051900','090430'],
          'Medicine' : ['207940','068270','302440','000100','128940'],
          'Non_Metal' : ['003670','003410','010780','300720'],
          'Metal': ['005490','010130', '004020','016380','001230'],
          'Machine' : ['034020','018880','241560','112610'],
          'Electronic' : ['005930','000660','373220','006400','066570'],
          'Construction' : ['000720','006360','047040','051600'],
          'Transport': ['011200','003490','086280','180640','028670'],
          'Distribution' : ['028260','023530','282330','139480','004170'],
          'Power': ['015760','036460','017390'],
          'Tele' : ['017670','030200','032640'],
          'Finance': ['323410','003550','000810','377300','006800'],
          'Brokerage' : ['005940','016360','008560','039490'],
          'Insurer':['032830','005830','000060','001450'],
          'Service':['035420','035720','259960','018260','036570'],
          'Manufacturer':['005380','000270','012330','329180','033780','009150']     
     }


In [6]:
df_sector = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}

# 3. 데이터 수집 함수

In [7]:
def checkStockPrice(sector_name,stockcode):
    dfs = [] #데이터프레임리스트
    count=0
    df = kiwoom.block_request("opt10081",
                          종목코드=stockcode,
                          기준일자=set_d,
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
    dfs.append(df)
    count+=1
    print('데이터 수집 시작.. ({}~)'.format(df.loc[0,'일자']))
    print('데이터 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
    while kiwoom.tr_remained:
        df = kiwoom.block_request("opt10081",
                                  종목코드=stockcode,
                                  기준일자=set_d,
                                  수정주가구분=1,
                                  output="주식일봉차트조회",
                                  next=2)
        dfs.append(df)
        time.sleep(1)
        print('데이터 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
        count+=1
        if kiwoom.tr_remained == False:
            print('데이터 수집 완료')
        elif count == 5:
            print('3000개 데이터를 확보')
            break;
        
    df_item = pd.concat(dfs)
    df_item.reset_index(drop=True,inplace=True)
    df_sector[sector_name][stockcode] = df_item

# 4. 데이터 수집 자동화

In [8]:
#데이터 수집 자동화 코드
for sector_name, stockcode_list in sector.items():
    print(f'---------{sector_name}업종 데이터 수집----------')
    for stockcode in stockcode_list:
        # 주식항목이름
        stockname = kiwoom.GetMasterCodeName(stockcode)
        print(f'--------[{stockname}] 항목 데이터 수집---------')
        checkStockPrice(sector_name,stockcode)
        
    

---------Food업종 데이터 수집----------
--------[CJ제일제당] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121005)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[오리온] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170707)
데이터 수집 완료
--------[하이트진로] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101


TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

데이터 수집 중.. (~20121005)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[농심] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121005)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[롯데칠성] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101


...

TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[현대모비스] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


TypeError: 'NoneType' object is not subscriptable

데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121005)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[현대중공업] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20210917)
--------[KT&G] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150317)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121005)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100511)
3000개 데이터를 확보
--------[삼성전기] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220707~)
데이터 수집 중.. (~20200205)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170818)
CommRqData opt10081 opt10081 2 0101
데이터 수집

# 5. 데이터 개수 확인
### 상장 일자 : 20100510 / 12년확보

In [9]:
# ('Food',['001680', '097950', '007310','271560','004370'])
for sector_name, stockcode_list in sector.items():
    for stockcode in stockcode_list:
        print('{} 수집된 데이터의 개수 : '.format(stockcode),end='')
        print(len(df_sector[sector_name][stockcode]))

097950 수집된 데이터의 개수 : 3000
271560 수집된 데이터의 개수 : 1229
000080 수집된 데이터의 개수 : 3000
004370 수집된 데이터의 개수 : 3000
005300 수집된 데이터의 개수 : 3000
383220 수집된 데이터의 개수 : 280
093050 수집된 데이터의 개수 : 3000
020000 수집된 데이터의 개수 : 3000
105630 수집된 데이터의 개수 : 3000
001070 수집된 데이터의 개수 : 3000
051910 수집된 데이터의 개수 : 3000
096770 수집된 데이터의 개수 : 3000
010950 수집된 데이터의 개수 : 3000
051900 수집된 데이터의 개수 : 3000
090430 수집된 데이터의 개수 : 3000
207940 수집된 데이터의 개수 : 1391
068270 수집된 데이터의 개수 : 3000
302440 수집된 데이터의 개수 : 324
000100 수집된 데이터의 개수 : 3000
128940 수집된 데이터의 개수 : 2944
003670 수집된 데이터의 개수 : 3000
003410 수집된 데이터의 개수 : 3000
010780 수집된 데이터의 개수 : 3000
300720 수집된 데이터의 개수 : 966
005490 수집된 데이터의 개수 : 3000
010130 수집된 데이터의 개수 : 3000
004020 수집된 데이터의 개수 : 3000
016380 수집된 데이터의 개수 : 3000
001230 수집된 데이터의 개수 : 3000
034020 수집된 데이터의 개수 : 3000
018880 수집된 데이터의 개수 : 3000
241560 수집된 데이터의 개수 : 1385
112610 수집된 데이터의 개수 : 1872
005930 수집된 데이터의 개수 : 3000
000660 수집된 데이터의 개수 : 3000
373220 수집된 데이터의 개수 : 108
006400 수집된 데이터의 개수 : 3000
066570 수집된 데이터의 개수 : 3000
000720 수집된 데이터의 

In [10]:
len(df_sector)

18

In [11]:
df_sector['Food']['271560'].columns

Index(['종목코드', '현재가', '거래량', '거래대금', '일자', '시가', '고가', '저가', '수정주가구분', '수정비율',
       '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가'],
      dtype='object')

In [12]:
df_sector['Food']['271560'].head()

,종목코드,현재가,거래량,거래대금,일자,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가
0,271560,107500,129398,13909,20220707,106000,108500,104500,,,,,,,
1,,105000,165771,17578,20220706,104000,109000,103000,,,,,,,
2,,105000,221429,22953,20220705,108000,108500,101500,,,,,,,
3,,108000,150928,16182,20220704,106500,109000,105000,,,,,,,
4,,106000,122573,12890,20220701,105500,106500,103500,,,,,,,


In [13]:
df_sector['Food']['271560'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229 entries, 0 to 1228
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   종목코드     1229 non-null   object
 1   현재가      1229 non-null   object
 2   거래량      1229 non-null   object
 3   거래대금     1229 non-null   object
 4   일자       1229 non-null   object
 5   시가       1229 non-null   object
 6   고가       1229 non-null   object
 7   저가       1229 non-null   object
 8   수정주가구분   1229 non-null   object
 9   수정비율     1229 non-null   object
 10  대업종구분    1229 non-null   object
 11  소업종구분    1229 non-null   object
 12  종목정보     1229 non-null   object
 13  수정주가이벤트  1229 non-null   object
 14  전일종가     1229 non-null   object
dtypes: object(15)
memory usage: 72.1+ KB


# 6. 특성선택

In [20]:
# df_sector = {'Food':{'code': DataFrame}}
for sector_name, df_dict in df_sector.items():
    for stockcode, df in df_dict.items():
        df = df[['일자','현재가','거래량']]
        df2 = pd.DataFrame()
        df2['일자'] = pd.to_datetime(df['일자'])
        df2['종가'] = df['현재가'].astype(int)
        df2['거래량'] = df['거래량'].astype(int)
        df_sector[sector_name][stockcode] = df2

# 7. 데이터 저장

In [22]:
import pickle
with open('df_sector.pickle','wb') as f:
    pickle.dump(df_sector,f)

In [23]:
with open('sector.pickle','wb') as f:
    pickle.dump(sector,f)

In [21]:
df_sector['Food']['271560']

,일자,종가,거래량
0,2022-07-07,107500,129398
1,2022-07-06,105000,165771
2,2022-07-05,105000,221429
3,2022-07-04,108000,150928
4,2022-07-01,106000,122573
...,...,...,...
1224,2017-07-13,81600,218138
1225,2017-07-12,83500,238407
1226,2017-07-11,82800,422303
1227,2017-07-10,88000,840451


In [24]:
df_sector['Food']['271560'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229 entries, 0 to 1228
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      1229 non-null   datetime64[ns]
 1   종가      1229 non-null   int32         
 2   거래량     1229 non-null   int32         
dtypes: datetime64[ns](1), int32(2)
memory usage: 19.3 KB


In [25]:
df_sector['Food']['271560']

,일자,종가,거래량
0,2022-07-07,107500,129398
1,2022-07-06,105000,165771
2,2022-07-05,105000,221429
3,2022-07-04,108000,150928
4,2022-07-01,106000,122573
...,...,...,...
1224,2017-07-13,81600,218138
1225,2017-07-12,83500,238407
1226,2017-07-11,82800,422303
1227,2017-07-10,88000,840451
